In [2]:
import requests
import pprint
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

url = "https://www.vgchartz.com/tools/hw_date.php?reg=Global&ending=Monthly"

headersStr = """accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: gzip, deflate, br
accept-language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6
cache-control: max-age=0
cookie: __qca=P0-1382307012-1662978577564; PHPSESSID=9cko4pq074ej9j4mhcremhp15u; geoCountry=US; uniqueFlag=1; VGCBE=b1; __utma=154313151.1784641658.1662978577.1662978577.1662978577.1; __utmb=154313151; __utmc=154313151; __utmz=154313151.1662978577.1.1.utmccn=(referral)|utmcsr=google.com|utmcct=/|utmcmd=referral; qcSxc=1662978577566; __qca=P0-1382307012-1662978577564; _pbjs_userid_consent_data=3524755945110770; _lr_retry_request=true; _lr_env_src_ats=false; cto_bundle=bX8Qp19MQ2Fjc250Q0FYM2FId094eXFLWlc4dkdiaCUyRlN4YVZQRzdyaEZmdHpwTk9qQ25SQzZaTWxMZkZ1THA1eENPU0UyRFBCWGgwT0VHMnRKeDhxaG56bUswTjF1T0ZaZ0tYT2VYT1RKRldNQ09DRDhZNjZnbklvdEVWeDRrWnpkc2tycGs2WW4zamdjck56REF0Y0RmMjc5QSUzRCUzRA; cto_bidid=bBeOA19NJTJCb0RpdEpNSGx0U2h0Nks4N05WVGpJSDdYbmRlVGF6V3FxRWRNOXklMkZ1YWVSJTJGYmtDYWtyd3hkVFlvRzE0Q0ZrMGIxOU43MzZHMk5UaE1KMEpIJTJCZHRra29Xa3BQUENlaTNES2NOQWFqbEMwJTNE
referer: https://www.vgchartz.com/tools/hw_date.php
sec-ch-ua: "Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"
sec-ch-ua-mobile: ?1
sec-ch-ua-platform: "Android"
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: same-origin
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36"""

headers = {r.split(": ")[0]: r.split(": ")[1] for r in headersStr.split("\n")}

res = requests.get(url, headers=headers)

In [3]:
res

<Response [200]>

In [4]:
soup = BeautifulSoup(res.text, "html.parser")

In [5]:
rawJsonObj = soup.select_one('[id="chart_body"] script').text
rawJsonObj = " ".join([i for i in rawJsonObj.split("\n") if "//" not in i])
rawJsonObj

' $(function() {         $(\'#reset_chart\').click(function() {             var extremes = chart.yAxis[0].getExtremes();             chart.yAxis[0].setExtremes(extremes.Min, extremes.Max);         });  \tHighcharts.setOptions({ \t\tchart: { \t\t\tstyle: { \t\t\t\tfontSize: \'11px\' \t\t\t} \t\t} \t});  \twindow.chart = new Highcharts.StockChart({ \t\tchart: { \t\t\trenderTo: \'chart_container\', \t\t\tmarginLeft: 80, \t\t\tmarginRight: 20, \t\t\tmarginBottom: 90, \t\t\tzoomType: \'y\', \t\t\tstartOnTick: false, \t\t\tendOnTick: false \t\t},  \t\ttitle: { \t\t\ttext: \'Monthly Hardware Comparisons - Global\' \t\t},  \t\tplotOptions: { \t\t\tseries: { \t\t\t\tdataGrouping: { \t\t\t\t\tsmoothed: true, \t\t\t\t} \t\t\t} \t\t},  \t\ttooltip: { \t\t\tstyle: { \t\t\t\tfontSize: \'11px\' \t\t\t} \t\t}, \t\t\t\t    \t\t\t\t     \t\txAxis: {                     \t\t\tshowLastLabel: true, \t\t\ttitle: { \t\t\t\ttext: \'Date\' \t\t\t} \t\t},  \t\tyAxis: { \t\t\ttitle: { \t\t\t\ttext: \'Monthly Uni

In [6]:
s = rawJsonObj \
    .split("window.chart = new Highcharts.StockChart(")[1] \
    .split(");")[0] \
    .replace("\n", " ") \
    .replace("\t", " ") \
    .replace("'", "\"") \
    .replace("{", "{ ") 
s

'{    chart: {     renderTo: "chart_container",    marginLeft: 80,    marginRight: 20,    marginBottom: 90,    zoomType: "y",    startOnTick: false,    endOnTick: false   },    title: {     text: "Monthly Hardware Comparisons - Global"   },    plotOptions: {     series: {      dataGrouping: {       smoothed: true,     }    }   },    tooltip: {     style: {      fontSize: "11px"    }   },                    xAxis: {                         showLastLabel: true,    title: {      text: "Date"    }   },    yAxis: {     title: {      text: "Monthly Units"    },    min: 0,    labels: {      align: "right",     x:-5    }   },    legend: {     enabled: true   },             series: [        {                           showLastLabel: true,            cropThreshold: 1,     name:"DS",     data: [     {          x: 1099285200000,         y: 546507     },     {          x: 1101877200000,         y: 2310880     },     {          x: 1104555600000,         y: 426958     },     {          x: 11072340000

In [7]:
keySet = set(re.findall(r'[a-zA-Z0-9]*:', s))

In [8]:
for kw in keySet:
    newKw = '"' + kw
    newKw = newKw.replace(':', '":')
#     print(newKw)
    s = s.replace(kw, newKw)
s

'{    "chart": {     "renderTo": "chart_container",    "marginLeft": 80,    "marginRight": 20,    "marginBottom": 90,    "zoomType": "y",    "startOnTick": false,    "endOnTick": false   },    "title": {     "text": "Monthly Hardware Comparisons - Global"   },    "plotOptions": {     "series": {      "dataGrouping": {       "smoothed": true,     }    }   },    "tooltip": {     "style": {      "fontSize": "11px"    }   },                    "xAxis": {                         "showLastLabel": true,    "title": {      "text": "Date"    }   },    "yAxis": {     "title": {      "text": "Monthly Units"    },    "min": 0,    "labels": {      "align": "right",     "x":-5    }   },    "legend": {     "enabled": true   },             "series": [        {                           "showLastLabel": true,            "cropThreshold": 1,     "name":"DS",     "data": [     {          "x": 1099285200000,         "y": 546507     },     {          "x": 1101877200000,         "y": 2310880     },     {    

In [9]:
jsonData = eval(
    s.replace("true", "True").replace("false", "False")
)

In [10]:
pprint.pprint(jsonData)

{'chart': {'endOnTick': False,
           'marginBottom': 90,
           'marginLeft': 80,
           'marginRight': 20,
           'renderTo': 'chart_container',
           'startOnTick': False,
           'zoomType': 'y'},
 'legend': {'enabled': True},
 'plotOptions': {'series': {'dataGrouping': {'smoothed': True}}},
 'series': [{'cropThreshold': 1,
             'data': [{'x': 1099285200000, 'y': 546507},
                      {'x': 1101877200000, 'y': 2310880},
                      {'x': 1104555600000, 'y': 426958},
                      {'x': 1107234000000, 'y': 290589},
                      {'x': 1109653200000, 'y': 919906},
                      {'x': 1112331600000, 'y': 448809},
                      {'x': 1114920000000, 'y': 383969},
                      {'x': 1117598400000, 'y': 496260},
                      {'x': 1120190400000, 'y': 381126},
                      {'x': 1122868800000, 'y': 492803},
                      {'x': 1125547200000, 'y': 584651},
                  

In [11]:
jsonData.keys()

dict_keys(['chart', 'title', 'plotOptions', 'tooltip', 'xAxis', 'yAxis', 'legend', 'series'])

In [12]:
jsonData["series"].__len__()

13

In [13]:
# Print all device names
for deviceInfo in jsonData["series"]:
    deviceName = deviceInfo["name"]
    print(deviceName)

DS
PS3
PSP
Wii
X360
3DS
PSV
WiiU
PS4
XOne
NS
XS
PS5


In [121]:
dfList = list()
for deviceInfo in jsonData["series"]:
    deviceName = deviceInfo["name"]
    data = deviceInfo["data"]
    
    df = pd.DataFrame(data=data)
    df.columns = ["timestamp_ms", "value"]
    
    df["device_name"] = deviceName
    df["datetime"] = df["timestamp_ms"].apply(lambda ts_ms: pd.to_datetime(ts_ms, unit='ms'))
    df["year"] = df["datetime"].apply(lambda dt: dt.year)
    df["month"] = df["datetime"].apply(lambda dt: dt.month)
    
    dfList.append(df)
#     df.to_csv(f"{deviceName}.csv", index=0, encoding='utf-8-sig')

df = pd.concat(dfList)

In [124]:
df[['device_name', 'timestamp_ms', 'value', 'datetime', 'year', 'month']].to_csv("all.csv", index=0, encoding="utf-8-sig")
df.columns

Index(['timestamp_ms', 'value', 'device_name', 'datetime', 'year', 'month'], dtype='object')

# TEST

In [13]:
soup.select_one('[id="chart_body"] script')

<script type="text/javascript">
$(function() {
        $('#reset_chart').click(function() {
            var extremes = chart.yAxis[0].getExtremes();
            chart.yAxis[0].setExtremes(extremes.Min, extremes.Max);
        });

	Highcharts.setOptions({
		chart: {
			style: {
				fontFamily: '"Lucida Grande", "Lucida Sans Unicode", Verdana, Arial, Helvetica, sans-serif', // default font
				fontSize: '11px'
			}
		}
	});

	// Create the chart	
	window.chart = new Highcharts.StockChart({
		chart: {
			renderTo: 'chart_container',
			marginLeft: 80,
			marginRight: 20,
			marginBottom: 90,
			zoomType: 'y',
			startOnTick: false,
			endOnTick: false
		},

		title: {
			text: 'Monthly Hardware Comparisons - Global'
		},

		plotOptions: {
			series: {
				dataGrouping: {
					smoothed: true,
				}
			}
		},

		tooltip: {
			style: {
				fontFamily: '"Lucida Grande", "Lucida Sans Unicode", Verdana, Arial, Helvetica, sans-serif', // default font
				fontSize: '11px'
			}
		},
				    				  

In [64]:
eval(s.replace("true", "True").replace("false", "False"))

{'chart': {'renderTo': 'chart_container',
  'marginLeft': 80,
  'marginRight': 20,
  'marginBottom': 90,
  'zoomType': 'y',
  'startOnTick': False,
  'endOnTick': False},
 'title': {'text': 'Monthly Hardware Comparisons - Global'},
 'plotOptions': {'series': {'dataGrouping': {'smoothed': True}}},
 'tooltip': {'style': {'fontSize': '11px'}},
 'xAxis': {'showLastLabel': True, 'title': {'text': 'Date'}},
 'yAxis': {'title': {'text': 'Monthly Units'},
  'min': 0,
  'labels': {'align': 'right', 'x': -5}},
 'legend': {'enabled': True},
 'series': [{'showLastLabel': True,
   'cropThreshold': 1,
   'name': 'X360',
   'data': [{'x': 1130821200000, 'y': 350847},
    {'x': 1133413200000, 'y': 828057},
    {'x': 1136091600000, 'y': 518934},
    {'x': 1138770000000, 'y': 331322},
    {'x': 1141189200000, 'y': 397650},
    {'x': 1143867600000, 'y': 483089},
    {'x': 1146456000000, 'y': 386471},
    {'x': 1149134400000, 'y': 405110},
    {'x': 1151726400000, 'y': 303676},
    {'x': 1154404800000, 'y

In [56]:
bqc = 0
mqc = 0
for idx, w in enumerate(s):
    if w == "{":
        bqc += 1
    if w == "}":
        bqc -= 1
    
    if w == "[":
        mqc += 1
    if w == "]":
        mqc -= 1
    
#     if bqc == 0:
#         print("bqc == 0", idx)
#     if mqc == 0:
#         print("mqc == 0", idx)
    
    if mqc == 0 and bqc == 0:
        print(idx)
        
print("bqc", bqc)
print("mqc", mqc)

78805
bqc 0
mqc 0


In [51]:
s[78805]

'}'

In [59]:
qc = 0
for idx, w in enumerate(s):
    if '"' in w:
        qc += 1
qc

5130

In [49]:
s[78805-15:78805+15]

'  ]       },]  }'

In [25]:
tests = '{    chart: {     renderTo: "chart_container" },  legend: {     enabled: true   } }'
re.sub(r'[a-zA-Z0-9]*:', "ttttt", tests)

'{    ttttt {     ttttt "chart_container" },  ttttt {     ttttt true   } }'

In [26]:
re.findall(r'[a-zA-Z0-9]*:', tests)

['chart:', 'renderTo:', 'legend:', 'enabled:']

In [28]:
set(re.findall(r'[a-zA-Z0-9]*:', s))

{'align:',
 'chart:',
 'cropThreshold:',
 'data:',
 'dataGrouping:',
 'enabled:',
 'endOnTick:',
 'fontSize:',
 'labels:',
 'legend:',
 'marginBottom:',
 'marginLeft:',
 'marginRight:',
 'min:',
 'name:',
 'plotOptions:',
 'renderTo:',
 'series:',
 'showLastLabel:',
 'smoothed:',
 'startOnTick:',
 'style:',
 'text:',
 'title:',
 'tooltip:',
 'x:',
 'xAxis:',
 'y:',
 'yAxis:',
 'zoomType:'}

In [84]:
datetime.fromtimestamp(1312171200)

datetime.datetime(2011, 8, 1, 12, 0)